In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import EvalPrediction,Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from datasets import Dataset

2024-05-08 09:30:33.827465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 09:30:33.827554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 09:30:33.948071: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
! pip install optuna

In [3]:
! pip install ray[tune]

In [13]:
df= pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  # 20% for testing
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

model_name = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)


In [6]:
MAX_LEN = 1024
def preprocess_function(examples):
    return tokenizer(examples['full_text'], padding="max_length", truncation=True, max_length=MAX_LEN)

In [14]:
def quadratic_weighted_kappa(y_true, y_pred):
    """
    Calculate the Quadratic Weighted Kappa (QWK aka Cohen's Kappa with quadratic weights).
    y_true: the ground truth labels
    y_pred: the predicted labels
    """
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "quadratic_weighted_kappa": quadratic_weighted_kappa(p.label_ids, preds),
    }

In [15]:
batch_size = 16
metric_name = "quadratic_weighted_kappa"
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

In [18]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
    }

In [20]:
trainer = Trainer(
    model=None,
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize",backend="optuna",hp_space=optuna_hp_space,)

[I 2024-05-08 09:42:21,271] A new study created in memory with name: no-name-e1fa38dc-c856-408a-8376-8756df04358e
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[W 2024-05-08 09:47:30,680] Trial 0 failed with parameters: {'learning_rate': 2.673227862289993e-05, 'per_device_train_batch_size': 16} because of the following error: IndexError('Invalid key: 12732 is out of bounds for size 0').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/opt/conda/lib/python3.10/site-packages/transformers/integrations/integration_utils.py", line 200, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/opt/conda/lib/python3.10/site-packages/transformers/trainer.py", line 1780, in train
    return inner_training_loop(
  File "/opt/conda/lib/python3.10/site-packages/transformers/trainer.py", line 2085, in _inner_training_loop
    for step, inputs in enumerate(epoch_iterator):
  File "/opt/conda/lib/python3.10/site-packages/accelerate/data_loader.py", line 452, in __iter__
    current_batch = next(datal

IndexError: Invalid key: 12732 is out of bounds for size 0